# User Guide: Processing data using OptionVolume

## OptionVolume_20190221

### _New Features and Changes(v2: 20190306)_
- Designed based on data file OptVol_20190221.xlsx


- Erase holiday, default erase_holiday = True


- Add weekly/monthly feature in __get_volume method:__
    - allow requesting cummulative option volume for 'W'(Weekly), 'M'(Monthly), or 'A'(both Weekly and Monthly), or None(default value, won't display Weekly/Monthly info)


- __get_tau() method:__
    - Input __OptVol__: dataframe, target Options that you want to calculate taus; 
    - Input __week_adj__: int/float, week adjustment for tau, 0 by default;
    - return __taus_df__: dataframe, Calculated tau for each date and each OptTicker.
        - taus_df.index = input OptVol.index
        - taus_df.colume = input OptVol.colume


- __get_volume_bytau() method:__
    - Input __OptVol__: dataframe, target Options that you want to formatted by taus;
    - Input __taus__: int/list, days prior to expiration;
    - return __tau_effect__: dataframe, contains OptVol for each tau and each OptTicker
        - tau_effect.index = input taus
        - tau_effect.colume = input OptVol.colume that already expired


- __concat_OptVolUnder_bytau() method: still testing__ 
    - Input __OptVol__: dataframe, target Options that you want to formatted by taus;
    - Input __Under__: dataframe, Underlying/Index data that has only 1 datapoint each date;
    - Input __taus__: int/list, days prior to expiration;
    - Input __data_columes__: list of str, colume you want to concate in output  and shift you wanna make
        - each str in the form of "colname(shifts)"
        - colname should be the element in input Under.colume
        - shifts could be either an int (1) or intergers separate by comma (1,3,4)
    - return __data_bytaus__: dataframe, Option volume and shifted Underlying data for each OptTicker and each tau
        - data_bytaus.index: OptTicekr + taus combination
        - data_bytaus.columes: corresponds to input data_columes
        

### _Incoming features_
- embed weekly/monthly expiration periodicity
--------------------------------------
### _New Features and Changes(v1: 20190227)_
- Designed based on data file OptVol_20190221.xlsx
    - Option Volume data range: 2015-01-01 to 2019-02-21, 5D rule
    - Option Ticker requestign datepoint: 2016-01-08 to 2019-02-21, length = 39 
    
    
- __get_volume method:__
    - allow requesting cummulative option volume for 'call' 'put' separately, by setting call_put = 'A'
    - allow automatically store requested data in excel, by setting export2excel = True, and excelname = 'Whatever name you want.xlsx'


- Option default Ticker delimiter: " " -> "_"
- Expiration data name rule: mm/dd/yy -> yyyymmdd

------------------------
## OptionVolume_20190222

### object setup

In [1]:
import pandas as pd
from OptionVolume_20190221 import OptionVolume # import OptionVolume in file OptionVolume_20190215.py
file_location = '../data/OptVol_20190221.xlsx'
workbook = pd.ExcelFile(file_location)

In [2]:
# create object that contains Option Volume
obj = OptionVolume(workbook,'AAPL')

In [3]:
obj.expdates_existed[-7:] # all expiration dates existed in data

['20190621',
 '20190719',
 '20191018',
 '20200117',
 '20200619',
 '20210115',
 '20210618']

### fetch Option Volume using get_volume method

In [4]:
df1 = obj.get_volume(expdates=['20190215','20190222']) # return daily volume that aggregated on all call/put strikes for 2 expiration dates
df2 = obj.get_volume(expdates=['20190215','20190222'],call_put='Call') # return call option daily volume that aggregated on all strikes for 2 expiration dates
df3 = obj.get_volume(expdates='20190222',call_put='P') # return put option daily volume that aggregated on all strikes for 02/22/19
df4 = obj.get_volume(expdates='20190222',call_put='A') # return call put option volume separately
df5 = obj.get_volume(call_put='A') # return call put option volume separately, won't provide Weekly/Monthly info
df6 = obj.get_volume(call_put='A',weeklys='W') # return Weekly expired option, and call put option volume separately
df7 = obj.get_volume(call_put='A',weeklys='A') # return all Weekly/Monthly expired option, and call put option volume separately
# df_ = obj.get_volume(call_put='A',export2excel=True,excelname='tmp_excelname.xlsx') # export user result to excel

In [15]:
df6.tail()

,AAPL_20160108_Call_W,AAPL_20160108_Put_W,AAPL_20160122_Call_W,AAPL_20160122_Put_W,AAPL_20160129_Call_W,AAPL_20160129_Put_W,AAPL_20160205_Call_W,AAPL_20160205_Put_W,AAPL_20160212_Call_W,AAPL_20160212_Put_W,...,AAPL_20190301_Call_W,AAPL_20190301_Put_W,AAPL_20190308_Call_W,AAPL_20190308_Put_W,AAPL_20190322_Call_W,AAPL_20190322_Put_W,AAPL_20190329_Call_W,AAPL_20190329_Put_W,AAPL_20190405_Call_W,AAPL_20190405_Put_W
2019-02-14,0,0,0,0,0,0,0,0,0,0,...,6031,2797,1811,970,1233,962,1574,600,0,0
2019-02-15,0,0,0,0,0,0,0,0,0,0,...,12758,6717,3148,1232,3444,1372,1500,780,0,0
2019-02-19,0,0,0,0,0,0,0,0,0,0,...,24331,11660,5272,1771,2472,1776,1021,852,0,0
2019-02-20,0,0,0,0,0,0,0,0,0,0,...,38645,21453,6069,2340,2156,1206,1728,1358,0,0
2019-02-21,0,0,0,0,0,0,0,0,0,0,...,36818,12051,6421,3071,1417,727,1074,745,723,86


In [17]:
df7.tail()

,AAPL_20160108_Call_W,AAPL_20160108_Put_W,AAPL_20160115_Call_M,AAPL_20160115_Put_M,AAPL_20160122_Call_W,AAPL_20160122_Put_W,AAPL_20160129_Call_W,AAPL_20160129_Put_W,AAPL_20160205_Call_W,AAPL_20160205_Put_W,...,AAPL_20191018_Call_M,AAPL_20191018_Put_M,AAPL_20200117_Call_M,AAPL_20200117_Put_M,AAPL_20200619_Call_M,AAPL_20200619_Put_M,AAPL_20210115_Call_M,AAPL_20210115_Put_M,AAPL_20210618_Call_M,AAPL_20210618_Put_M
2019-02-14,0,0,0,0,0,0,0,0,0,0,...,0,0,5962,5895,801,160,591,560,207,36
2019-02-15,0,0,0,0,0,0,0,0,0,0,...,0,0,4380,4052,1004,191,712,201,337,467
2019-02-19,0,0,0,0,0,0,0,0,0,0,...,544,950,6416,3387,1668,1793,1238,433,265,979
2019-02-20,0,0,0,0,0,0,0,0,0,0,...,528,4626,3792,5746,324,1278,2317,1896,578,625
2019-02-21,0,0,0,0,0,0,0,0,0,0,...,183,1133,3337,7488,444,205,1057,911,298,330


### calculate $\tau$ for the specific OptVol dataframe

In [5]:
# first request OptVol dataframe
allOptTickers = obj.get_volume(call_put='A',weeklys='A')
# get corresponding tau for this OptVol dataframe
taus_df = obj.get_tau(allOptTickers, week_adj=0.1)

In [6]:
taus_df['2016'].head()

,AAPL_20160108_Call_W,AAPL_20160108_Put_W,AAPL_20160115_Call_M,AAPL_20160115_Put_M,AAPL_20160122_Call_W,AAPL_20160122_Put_W,AAPL_20160129_Call_W,AAPL_20160129_Put_W,AAPL_20160205_Call_W,AAPL_20160205_Put_W,...,AAPL_20191018_Call_M,AAPL_20191018_Put_M,AAPL_20200117_Call_M,AAPL_20200117_Put_M,AAPL_20200619_Call_M,AAPL_20200619_Put_M,AAPL_20210115_Call_M,AAPL_20210115_Put_M,AAPL_20210618_Call_M,AAPL_20210618_Put_M
2016-01-04,4,4,9.1,9.1,13.2,13.2,18.3,18.3,23.4,23.4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-01-05,3,3,8.1,8.1,12.2,12.2,17.3,17.3,22.4,22.4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-01-06,2,2,7.1,7.1,11.2,11.2,16.3,16.3,21.4,21.4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-01-07,1,1,6.1,6.1,10.2,10.2,15.3,15.3,20.4,20.4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-01-08,0,0,5.1,5.1,9.2,9.2,14.3,14.3,19.4,19.4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### fetch option volume categoried by tau using get_volume_bytau method

In [7]:
Weeklys = obj.get_volume(weeklys='W')
Monthlys = obj.get_volume(weeklys='M')

tau_effect_W = obj.get_volume_bytau(Weeklys, taus=range(5))
tau_effect_M = obj.get_volume_bytau(Monthlys, taus=range(5))

In [8]:
tau_effect_W

,AAPL_20160108_W,AAPL_20160122_W,AAPL_20160129_W,AAPL_20160205_W,AAPL_20160212_W,AAPL_20160226_W,AAPL_20160304_W,AAPL_20160311_W,AAPL_20160324_W,AAPL_20160401_W,...,AAPL_20181123_W,AAPL_20181130_W,AAPL_20181207_W,AAPL_20181214_W,AAPL_20181228_W,AAPL_20190104_W,AAPL_20190111_W,AAPL_20190125_W,AAPL_20190201_W,AAPL_20190208_W
0,348795,172237,81493,103934,166853,225184,323980,162681,166844,213226,...,156961,284287,303296,269470,311102,353866,186470,288808,193256,205250
1,315611,66630,88814,110591,225935,148148,214276,216778,140363,116356,...,212006,320637,343003,231318,357590,420292,243673,144208,304827,393088
2,301188,100416,293424,92280,150088,181155,171673,125064,178183,230733,...,361880,332885,271404,192042,297016,156136,306195,166103,436951,191231
3,232683,75558,421956,64164,170932,164789,333361,117350,225392,200911,...,230049,236128,258347,243929,153455,156506,246642,217306,368458,274871
4,252738,96055,215105,77149,209564,162293,162054,174900,88256,111194,...,128162,277010,131149,351340,169767,114376,192571,129740,142327,260103


In [9]:
tau_effect_M

,AAPL_20160115_M,AAPL_20160219_M,AAPL_20160318_M,AAPL_20160415_M,AAPL_20160520_M,AAPL_20160617_M,AAPL_20160715_M,AAPL_20160819_M,AAPL_20160916_M,AAPL_20161021_M,...,AAPL_20180518_M,AAPL_20180615_M,AAPL_20180720_M,AAPL_20180817_M,AAPL_20180921_M,AAPL_20181019_M,AAPL_20181116_M,AAPL_20181221_M,AAPL_20190118_M,AAPL_20190215_M
0,468614,300562,260908,558249,319610,324949,195034,192020,571890,184519,...,176700,242967,135820,351257,328210,361469,224658,182107,124277,190436
1,454294,267190,234715,193111,243510,276871,322407,134629,637380,230183,...,171166,154285,160709,271785,243303,303878,238648,195631,166626,176824
2,361827,295557,245690,277969,303138,198109,85810,163291,860669,156134,...,168229,150625,106142,215937,229923,192631,344452,211037,228420,189833
3,249916,262564,265249,202890,230056,213309,111254,188642,375276,198909,...,190085,112869,132674,148569,247351,258446,264889,187460,98025,128901
4,246297,142984,145816,224530,453236,232030,134640,147495,252021,199029,...,163681,138244,111872,264007,288741,234207,349109,178441,90549,195426


### concatenate OptVol with Underlying using concat_OptVolUnder_bytau method

In [10]:
Calls = obj.get_volume(call_put='C')
Puts = obj.get_volume(call_put='P')

In [11]:
# read underlying data
Underlying = pd.read_excel('../data/AAPL_SP_DATA.xlsx',sheet_name='Sheet3',header=0,skiprows=1,index_col=0)

In [12]:
CallsUnder_bytaus = obj.concat_OptVolUnder_bytau(Calls,Underlying,taus=3,
        data_columes=['OptVol','CHG_PCT_1D','PX_VOLUME(1,2,3,4)'])

PutsUnder_bytaus = obj.concat_OptVolUnder_bytau(Puts,Underlying,taus=range(5),
        data_columes=['PX_VOLUME(1,4)','VOLATILITY_180D(1)'])


In [14]:
CallsUnder_bytaus.head(10)

,OptVol,CHG_PCT_1D,PX_VOLUME(1),PX_VOLUME(2),PX_VOLUME(3),PX_VOLUME(4)
AAPL_20160108_C_tau3,130817,-2.5059,67649387.0,40912316.0,25213777.0,30931243.0
AAPL_20160115_C_tau3,131386,1.4513,49739377.0,70798016.0,81094428.0,68457388.0
AAPL_20160122_C_tau3,54800,-0.4839,79833891.0,63170127.0,62439631.0,49154227.0
AAPL_20160129_C_tau3,277919,0.5531,51794525.0,65800467.0,52161463.0,72334416.0
AAPL_20160205_C_tau3,47527,-2.0222,40943541.0,64416504.0,55678825.0,133369674.0
AAPL_20160212_C_tau3,86179,-0.0211,54021375.0,46418064.0,46471652.0,45964294.0
AAPL_20160219_C_tau3,152765,2.8194,40351381.0,50074711.0,42343601.0,44331195.0
AAPL_20160226_C_tau3,83286,-2.2605,34280758.0,35374173.0,39020983.0,44863243.0
AAPL_20160304_C_tau3,198580,3.9715,35216277.0,28991131.0,27582659.0,36255745.0
AAPL_20160311_C_tau3,63759,-0.8246,35915810.0,46055100.0,36955742.0,33169560.0


In [16]:
PutsUnder_bytaus.head(10)

,OptVol,PX_VOLUME(1),PX_VOLUME(4),VOLATILITY_180D(1)
AAPL_20160108_P_tau0,183191,81094428.0,67649387.0,28.038
AAPL_20160108_P_tau1,132679,68457388.0,40912316.0,27.608
AAPL_20160108_P_tau2,138634,55790992.0,25213777.0,27.574
AAPL_20160108_P_tau3,101866,67649387.0,30931243.0,27.423
AAPL_20160108_P_tau4,113460,40912316.0,26704210.0,27.570
AAPL_20160115_P_tau0,205536,63170127.0,49739377.0,28.106
AAPL_20160115_P_tau1,189117,62439631.0,70798016.0,28.145
AAPL_20160115_P_tau2,166535,49154227.0,81094428.0,28.034
AAPL_20160115_P_tau3,118530,49739377.0,68457388.0,28.021
AAPL_20160115_P_tau4,121619,70798016.0,55790992.0,28.040


------------------------
## OptionVolume_20190215

In [18]:
import pandas as pd
from OptionVolume_20190215 import OptionVolume # import OptionVolume in file OptionVolume_20190215.py
file_location = '../data/template_20190215.xlsx'
workbook = pd.ExcelFile(file_location)

In [19]:
obj = OptionVolume(workbook,'AAPL')
df1 = obj.get_volume(expdates=['02/15/19','02/22/19']) # return daily volume that aggregated on all call/put strikes for 2 expiration dates
df2 = obj.get_volume(expdates=['02/15/19','02/22/19'],call_put='Call') # return call option daily volume that aggregated on all strikes for 2 expiration dates
df3 = obj.get_volume(expdates='02/22/19',call_put='P') # return put option daily volume that aggregated on all strikes for 02/22/19
df4 = obj.get_volume(call_put = 'Put') # return put option daily volume that aggregated on all strikes for all existed expiration dates

In [20]:
obj.expdates_existed # all expiration dates existed in data

['03/08/19',
 '02/22/19',
 '03/22/19',
 '03/15/19',
 '03/29/19',
 '06/18/21',
 '07/19/19',
 '04/18/19',
 '03/01/19',
 '06/19/20',
 '01/17/20',
 '01/15/21',
 '02/15/19',
 '06/21/19']

In [21]:
df1.iloc[-5:]

,AAPL_02/15/19,AAPL_02/22/19
Dates,,
2019-02-11,197570.0,23460.0
2019-02-12,130102.0,32205.0
2019-02-13,191659.0,41509.0
2019-02-14,177168.0,53513.0
2019-02-15,190436.0,83582.0


In [22]:
df2.iloc[-5:]

,AAPL_02/15/19_Call,AAPL_02/22/19_Call
Dates,,
2019-02-11,84601.0,16236.0
2019-02-12,84327.0,23076.0
2019-02-13,89176.0,27858.0
2019-02-14,108604.0,34046.0
2019-02-15,109342.0,51014.0


In [23]:
df3.iloc[-5:]

,AAPL_02/22/19_P
Dates,
2019-02-11,7224.0
2019-02-12,9129.0
2019-02-13,13651.0
2019-02-14,19467.0
2019-02-15,32568.0


In [24]:
df4.iloc[-5:]

,AAPL_03/08/19_Put,AAPL_02/22/19_Put,AAPL_03/22/19_Put,AAPL_03/15/19_Put,AAPL_03/29/19_Put,AAPL_06/18/21_Put,AAPL_07/19/19_Put,AAPL_04/18/19_Put,AAPL_03/01/19_Put,AAPL_06/19/20_Put,AAPL_01/17/20_Put,AAPL_01/15/21_Put,AAPL_02/15/19_Put,AAPL_06/21/19_Put
Dates,,,,,,,,,,,,,,
2019-02-11,2718.0,7224.0,1009.0,30366.0,532.0,1672.0,797.0,22116.0,2291.0,226.0,3107.0,1056.0,112969.0,3084.0
2019-02-12,775.0,9129.0,413.0,21192.0,748.0,91.0,2036.0,4158.0,2305.0,1135.0,1888.0,267.0,45775.0,8310.0
2019-02-13,995.0,13651.0,570.0,22212.0,324.0,300.0,1203.0,20441.0,2268.0,913.0,4359.0,2810.0,102483.0,7404.0
2019-02-14,970.0,19467.0,962.0,11901.0,600.0,36.0,4707.0,3847.0,2794.0,155.0,5895.0,556.0,68564.0,6602.0
2019-02-15,1231.0,32568.0,1371.0,15935.0,780.0,461.0,621.0,8485.0,6703.0,185.0,4032.0,198.0,81094.0,2658.0


In [25]:
df5 = obj.get_volume(expdates='02/25/19') # raise Error if expdates not exist

ValueError: No  data expired at 02/25/19 was found, formatting option volume failed